# Llama 3

Llama 3 8B (instruction-tuned) outperforms Gemma 7B and Mistral 7B Instruct. Llama 3 70 broadly outperforms Gemini Pro 1.5 and Claude 3 Sonnet and falls a bit behind on the MATH benchmark when compared to Gemini Pro 1.5.

In [1]:
import pandas as pd
df = pd.read_csv("../data/cleaned_patient_data.csv")
df["text"] = df["ANA"] + ". " + df["EXA"]

## German Llama 3 no architectural changes, no fine-tuning, no pre-trained tokenizer, default hyperparameteres

### Text generation with 0-shot prompting (No prompt engineering)

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    "DiscoResearch/Llama3-DiscoLeo-Instruct-8B-v0.1",
    torch_dtype="auto"
)
tokenizer = AutoTokenizer.from_pretrained("DiscoResearch/Llama3-DiscoLeo-Instruct-8B-v0.1")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
prompt = df["text"][0] + " Die Diagnose ist: "
print(prompt)

Beschwerden re. Achillessehne. NMR unauffällig. Verdickung jedoch da; nach und während des Laufen (Sprint) schlimmer.. Re. Achillessehne: Verdickung im Bereich der Achillessehne ca. 3 cm proximal; Re. Leiste: Druckschmerz am os pubis zum os ischii hin; auch nach ventral am os pubis Die Diagnose ist: 


In [5]:
messages = [
    {"role": "system", "content": "Du bist ein Arzt, der von einem Patienten eine Beschreibung der Symptome eines Knieproblems erhält. Du musst den Patienten diagnostizieren und diese Diagnose vorhersagen. Gib nur die endgültige Diagnose aus."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt")

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [6]:
response

'Achillessehnenentzündung.'

### Text generation with few-shot prompting

In [7]:
prompt = df["text"][0] + " Ausgabe: "
print(prompt)

Beschwerden re. Achillessehne. NMR unauffällig. Verdickung jedoch da; nach und während des Laufen (Sprint) schlimmer.. Re. Achillessehne: Verdickung im Bereich der Achillessehne ca. 3 cm proximal; Re. Leiste: Druckschmerz am os pubis zum os ischii hin; auch nach ventral am os pubis Ausgabe: 


In [8]:
df["DIA_code"] = df["DIA_code"].apply(lambda x: x.split(" ")[0])
unique_df = df.drop_duplicates(subset=["DIA_code"], keep='first')

In [9]:
unique_df

,Unnamed: 0,patientid,sex,age,ANA,EXA,DIA_text,DIA_code,OP_text,OP_code,text
0,1,61.0,male,32.0,Beschwerden re. Achillessehne. NMR unauffällig...,Re. Achillessehne: Verdickung im Bereich der A...,Retropatellare Chondromalazie li; Insertionste...,M22.4,NaN,NaN,Beschwerden re. Achillessehne. NMR unauffällig...
1,2,67.0,male,41.0,Seit mehreren Monaten Beschwerden re. Achilles...,Re. Achillessehne: Deutliche Auftreibung der A...,V.a.laterale Chondromalazie bei st.n. Außenmen...,M23.3-,NaN,NaN,Seit mehreren Monaten Beschwerden re. Achilles...
3,7,72.0,male,23.0,beide Knie Schmerzen bei Treppe steigen oder L...,bd. Knie: frei Bewegl.; Patella o.B.; keine MZ...,Z.n.IM-Teilresektion re.,S83.2,NaN,NaN,beide Knie Schmerzen bei Treppe steigen oder L...
4,9,84.0,male,57.0,(08.19 Uhr); bewegungsabhängig.Schmerzen wiede...,Klinischer Befund des Kniegelenks li.: Drucksc...,Beginnende Gonarthrose bds. li. mehr als re.,M17.9,NaN,NaN,(08.19 Uhr); bewegungsabhängig.Schmerzen wiede...
6,13,94.0,female,62.0,(09.24 Uhr) Seit mehreren Monaten Hüftbeschwer...,Klinischer Befund des Hüftgelenks li.: pos. Tr...,Sonstige Meniskusschädigungen: Hinterhorn des ...,M23.32,NaN,NaN,(09.24 Uhr) Seit mehreren Monaten Hüftbeschwer...
...,...,...,...,...,...,...,...,...,...,...,...
31816,37462,100433.0,male,49.0,Vor 5 Jahren AC re.Knie mit IM-VH-Telresektion...,flüssiges symmetrisches GB; varische BA 2 QF r...,Binnenschädigung des Kniegelenkes; nicht näher...,M23.94,NaN,NaN,Vor 5 Jahren AC re.Knie mit IM-VH-Telresektion...
34859,40651,108305.0,male,38.0,Seit 2 Jahren Schmerzen im re. Knie im Bereich...,Ca. 2 QF Valgisch; flüssiges Gangbild; mäßiger...,Meniskusganglion: sonstiger und nicht näher be...,M23.03,NaN,NaN,Seit 2 Jahren Schmerzen im re. Knie im Bereich...
36304,42136,110752.0,female,63.0,Vorstellung mit Beschwerden im linken Knie sei...,Gangbild flüßig; Beinachse 2QF varisch; Einbei...,Meniskusganglion: Hinterhorn des Außenmeniskus...,M23.05,NaN,NaN,Vorstellung mit Beschwerden im linken Knie sei...
38933,44801,302217.0,female,14.0,Probleme Knie beide Seite; MRT heute rechte Se...,ROM bds frei; kein DS bds; keine Meniskuszeich...,Scheibenmeniskus (angeboren) rechts,M23.1-,NaN,NaN,Probleme Knie beide Seite; MRT heute rechte Se...


In [10]:
shot_1 = unique_df["text"].iloc[1]
shot_1_output = unique_df["DIA_text"].iloc[1]

shot_2 = unique_df["text"].iloc[2]
shot_2_output = unique_df["DIA_text"].iloc[2]


shot_3 = unique_df["text"].iloc[3]
shot_3_output = unique_df["DIA_text"].iloc[3]

In [11]:
messages = [
    {"role": "system", "content":
     """
     Du bist ein Arzt, der von einem Patienten eine Beschreibung der Symptome eines Knieproblems erhält. Du musst den Patienten diagnostizieren und diese Diagnose vorhersagen. Gib nur die endgültige Diagnose aus.

     Patient: ${shot_1}
     Ausgabe: ${shot_1_output}

     Patient: ${shot_2}
     Ausgabe: ${shot_2_output}

     Patient: ${shot_3}
     Ausgabe: ${shot_3_output}
     """},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt")

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [12]:
response

'Verdacht auf Achillodynie rechts.'